In [1]:
#coding:utf-8
import pandas as pd
import numpy as np
import time
import datetime
import gc
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

In [34]:
path = './data/'
data = pd.read_csv(path+"all_final_data_11-25-20-35.csv")


In [6]:
len(data.columns)

302

In [36]:
train = pd.read_csv(path+"train_all.csv")
train = train[['instance_id']]
train

,instance_id
0,5.089880e+18
1,2.163600e+18
2,8.229730e+18
3,8.978460e+18
4,3.831630e+18
...,...
79263,1.000380e+18
79264,9.126350e+18
79265,1.583620e+18
79266,4.525820e+18


In [37]:
train =pd.merge(train, data, how='left', on='instance_id')

In [10]:
train.head()

,instance_id,item_id,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,...,large2,shop_id_query_day,category_2_query_day,price_diff_7_6,price_diff_7_5,price_diff_7_4,user_till_now_cnt,shop_till_now_cnt,item_till_now_cnt,brand_till_now_cnt
0,5.089880e+18,2769,5977512434884267894;2072967855524022579;513128...,5982,2,6,11,10,17,60637,...,0,10,7448,6.0,NaN,NaN,0.0,0.0,0.0,0.0
1,2.163600e+18,16242,6241534230954727302;367082587220462692;5977512...,4705,147,6,9,10,16,33155,...,0,46,7027,6.0,6.0,6.0,0.0,0.0,0.0,0.0
2,8.229730e+18,7731,5977512434884267894;2636395404473730413;462193...,5385,80,4,12,12,13,38351,...,0,5,7661,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3,8.978460e+18,68,367082587220462692;5977512434884267894;2072967...,3543,128,9,4,5,11,91026,...,0,1,7448,NaN,NaN,NaN,0.0,0.0,0.0,0.0
4,3.831630e+18,556,2072967855524022579;639228713552738893;3163265...,6156,103,5,11,9,14,55740,...,0,2,7027,NaN,5.0,NaN,0.0,0.0,0.0,0.0


In [46]:
def feat_select(train, test):
  
    features = train.drop(['is_trade', 'time', 'context_timestamp','item_property_list','predict_category_property'], axis=1).columns.tolist()

    target = ['is_trade']

    return features, target

In [50]:
def lgbCV(train, test, All):
    features, target = feat_select(train, test)

    lgb_clf = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=48, max_depth=-1, learning_rate=0.05)
    # lgb_clf = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=48, max_depth=-1, learning_rate=0.05, n_estimators=2000,
    #                            max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
    #                            min_child_weight=5, min_child_samples=10, subsample=1, subsample_freq=1,
    #                            colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=1000, nthread=-1, silent=True)

    lgb_model = lgb_clf.fit(train[features], train[target], eval_set=[(test[features], test[target])], early_stopping_rounds=200)
    best_iter = lgb_model.best_iteration_ 
    
    # 特征重要性
    lgb_predictors = [i for i in train[features].columns]
    lgb_feat_imp = pd.Series(lgb_model.feature_importances_, lgb_predictors).sort_values(ascending=False)
    lgb_feat_imp.to_csv('lgb_feat_imp.csv')
    
    # 训练模型
    lgb_clf.fit(train[features], train[target])
    test['lgb_predict'] = lgb_clf.predict_proba(test[features])[:, 1]
    lgb_loss = log_loss(test[target], test['lgb_predict'])  
    lgb_auc = roc_auc_score(test[target], test['lgb_predict'])
    print('Training loss: %.6f, Training AUC: %.6f' % (lgb_loss, lgb_auc))
    
    return best_iter,lgb_loss

In [40]:
test = pd.read_csv(path+"test_all.csv")
test = test[['instance_id']]
test = pd.merge(test, data, how='left', on='instance_id')

In [51]:
All = True
best_iter, lgb_loss = lgbCV(train, test, All)

c:\Users\HP\Downloads\Python39\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\HP\Downloads\Python39\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\HP\Downloads\Python39\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's binary_logloss: 0.0708301
[2]	valid_0's binary_logloss: 0.0703195
[3]	valid_0's binary_logloss: 0.0698135
[4]	valid_0's binary_logloss: 0.0692581
[5]	valid_0's binary_logloss: 0.0688738
[6]	valid_0's binary_logloss: 0.0685225
[7]	valid_0's binary_logloss: 0.0682439
[8]	valid_0's binary_logloss: 0.0679741
[9]	valid_0's binary_logloss: 0.0677871
[10]	valid_0's binary_logloss: 0.0675866
[11]	valid_0's binary_logloss: 0.0673863
[12]	valid_0's binary_logloss: 0.0672525
[13]	valid_0's binary_logloss: 0.0671075
[14]	valid_0's binary_logloss: 0.0669432
[15]	valid_0's binary_logloss: 0.0667432
[16]	valid_0's binary_logloss: 0.0666559
[17]	valid_0's binary_logloss: 0.0665537
[18]	valid_0's binary_logloss: 0.0664919
[19]	valid_0's binary_logloss: 0.0663786
[20]	valid_0's binary_logloss: 0.0663519
[21]	valid_0's binary_logloss: 0.0663309
[22]	valid_0's binary_logloss: 0.0662896
[23]	valid_0's binary_logloss: 0.0661982
[24]	valid_0's binary_logloss: 0.066177
[25]	valid_0's binary_logl

c:\Users\HP\Downloads\Python39\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\HP\Downloads\Python39\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training loss: 0.065293, Training AUC: 0.775519


In [52]:
best_iter

73

In [53]:
lgb_loss

0.06529261841333364